<a href="https://colab.research.google.com/github/yp6969/Object_detection_Drone/blob/main/human_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Human classification convulution learning

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###checking if using GPU

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Thu Jun  9 18:32:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import os
from typing import List, Tuple

import imageio
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image
import random

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow import keras
from keras.layers.core import (
    Activation,
    Reshape,
    Flatten,
    Dense,
)
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D

In [4]:
%cd /content/drive/MyDrive/human_detection_project/Database/human_detection_dataset

/content/drive/MyDrive/human_detection_project/Database/human_detection_dataset


In [5]:
%ls

human/  non_human/


In [7]:
IMAGE_SHAPE = (256, 256, 3)

##load images to people and no_people lists

In [8]:
people_path = "human/"
no_people_path = "non_human/"
people = []
no_people = []
range_t = [i for i in range(231)]

people_dir = os.listdir(people_path)
no_people_dir = os.listdir(no_people_path)

for i in tqdm(range_t):
    people.append(Image.open(f"{people_path}{people_dir[i]}"))

for i in tqdm(range_t):
    no_people.append(Image.open(f"{no_people_path}{no_people_dir[i]}"))

100%|██████████| 231/231 [00:55<00:00,  4.19it/s]


In [9]:
def reshape_images(images):
  """
  reshape the images that is not in shape: (256, 256, 3)
  """
  reshaped_images = []
  for i, img in enumerate(images):
      if img.size != (256, 256):
          img = img.resize((256, 256))
      new_img = np.asarray(img)
      new_img = new_img.astype(np.float32)
      if new_img.shape == IMAGE_SHAPE:
          reshaped_images.append(new_img)
  return reshaped_images

In [10]:
people_images = reshape_images(people)
no_people_images = reshape_images(no_people)

In [11]:
def check_shape(img_lst: list):
  for i, img in enumerate(img_lst):
    assert img.shape == IMAGE_SHAPE, f"error with image number {i}"

chack shape of the images

In [12]:
check_shape(people_images)
check_shape(no_people_images)

## lable the images
1 to people
0 to non people

In [13]:
shuffled_data = [(img, 1.0) for img in people_images]
shuffled_data.extend([(img, 0.0) for img in no_people_images])

In [14]:
# shuffle the images
random.shuffle(shuffled_data)

##normalize the images to rgb range [0-1]

In [15]:
norm = lambda mat: ((mat[0] - mat[0].min()) / mat[0].max(), mat[1])

shuffled_data = map(norm, shuffled_data)
shuffled_data = list(shuffled_data)

##define imaged and lable lists

In [16]:
images = []
labels = []

for img, label in shuffled_data:
  images.append(img)
  labels.append(label)

assert len(images) == len(labels), "labels and images should have the same length"

images = np.array(images)
labels = np.array(labels)

##data spliting

In [17]:
n = len(images)
split = int(n * 0.7)

train_images = images[: split]
train_labels = labels[: split]
test_images = images[split:]
test_labels = labels[split:]

# val_images = train_images[int(0.9*len(train_images)):]
# val_labels = train_labels[int(0.9*len(train_images)):]

n

410

#Define the Model

##custom model

In [18]:
def model_generator():
    IMAGE_SIZE = 256
    RGB = 3
    NFILTERS = 8
    CONV_PARAMS = {"kernel_size": 3, "use_bias": False, "padding": "same"}

    model = Sequential()
    # model.add(Reshape((64, IMAGE_SIZE, IMAGE_SIZE, RGB), input_shape=(IMAGE_SIZE, IMAGE_SIZE, RGB)))
    model.add(Reshape((IMAGE_SIZE, IMAGE_SIZE, RGB)))

    # adding the model layers
    for i in [1, 2, 4, 8, 16, 32, 64]:
        model.add(Conv2D(NFILTERS * i, **CONV_PARAMS))
        model.add(BatchNormalization(momentum=0.1))
        model.add(Activation("relu"))
        model.add(MaxPooling2D())

    model.add(Flatten())
    model.add(Dense(64, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    return model

In [34]:
def trainer(model: Sequential, images, labels, epochs=10):
    BATCH_SIZE = 64
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics='accuracy')  # adam = gradient descent algorithm
    history = model.fit(x=images, y=labels, batch_size=BATCH_SIZE, verbose=1, epochs=epochs, validation_split=0.1)
    model.save("model3.hdf5")

##evaluate the model and return the prediction

In [20]:
def eval(model, test_x: np.array, test_y: np.array) -> Tuple[np.array, float]:
    for layer in model.layers:
      if "batch_normalization" in layer.name:
        layer.trainable = False
        # print("weigths:", len(layer.weights))
        # print("trainable_weights:", len(layer.trainable_weights))
        # print("non_trainable_weights:", len(layer.non_trainable_weights))
    # model.summary()
    threshhold = 0.5
    predicted_y = []
    for i, img in enumerate(test_x):
        pred = model.predict(img[None])
        pred = 1 if pred[0][0] > threshhold else 0
        predicted_y.append(pred)

    accuracy = sum([1 if predicted_y[i] == int(test_y[i]) else 0 for i in range(len(predicted_y))]) / len(predicted_y)
    with open("/content/drive/MyDrive/human_detection_project/results/classification_results.txt", "w") as f:
        f.write(f"this is my prediction {predicted_y}\n")
        f.write(f"this is True {test_y}\n")
        f.write(f"accuracy score: {accuracy}\n")
    return np.array(predicted_y), accuracy

In [21]:
def plot_images(images: List[np.ndarray],
                preds: np.ndarray,
                labels: np.ndarray,
                size: Tuple[int, int] = (4, 4)) -> None:
    f, axs = plt.subplots(size[0], size[1], figsize=(4 * size[0], 4 * size[1]))
    for img, p, label, ax in zip(images, preds, labels, axs.ravel()):
        ax.imshow(img)
        ax.set_title(f"Pred: {p}, Actual: {label}")
        ax.axis('off')
    plt.show("prediction.png")

In [22]:
%cd /content/drive/MyDrive/human_detection_project/models
%ls

/content/drive/MyDrive/human_detection_project/models
model1.hdf5  transfer_learning_mobileNet2.hdf5
model2.hdf5  transfer_learning_mobileNet.hdf5


##Generate the model

In [38]:
# data sets:  train_images, train_labels, test_images, test_labels
# 291 train images, 126 test images

model = model_generator()
history = trainer(model, train_images, train_labels, epochs=30)
history

Epoch 1/30
5/5 [==============================] - 2s 146ms/step - loss: 2.0039 - accuracy: 0.6163 - val_loss: 0.8391 - val_accuracy: 0.7931
Epoch 2/30
5/5 [==============================] - 0s 88ms/step - loss: 0.8232 - accuracy: 0.6744 - val_loss: 0.4925 - val_accuracy: 0.8621
Epoch 3/30
5/5 [==============================] - 0s 96ms/step - loss: 0.4923 - accuracy: 0.7287 - val_loss: 0.4536 - val_accuracy: 0.8621
Epoch 4/30
5/5 [==============================] - 0s 88ms/step - loss: 0.4855 - accuracy: 0.7481 - val_loss: 0.8893 - val_accuracy: 0.7586
Epoch 5/30
5/5 [==============================] - 0s 97ms/step - loss: 0.4842 - accuracy: 0.7403 - val_loss: 0.3789 - val_accuracy: 0.7931
Epoch 6/30
5/5 [==============================] - 0s 86ms/step - loss: 0.4235 - accuracy: 0.7791 - val_loss: 0.4162 - val_accuracy: 0.8276
Epoch 7/30
5/5 [==============================] - 0s 88ms/step - loss: 0.4187 - accuracy: 0.7597 - val_loss: 1.2255 - val_accuracy: 0.7931
Epoch 8/30
5/5 [==========

In [39]:
prediction, accuracy = eval(model, test_images, test_labels)

In [24]:
model_t = keras.models.load_model("model1.hdf5")
prediction, accuracy = eval(model_t, test_images, test_labels)

In [25]:
prediction

array([0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0])

In [26]:
accuracy

0.943089430894309

In [ ]:
# model = keras.models.load_model("model1.hdf5")
# results = model_t.evaluate(test_images, test_labels, batch_size=12)
# print("test loss, test acc:", results)
# predicted_lable, accuracy = eval(test_images, test_labels)
# plot_images(test_images, predicted_lable, test_labels)

# pred = model_t.predict(train_images[10][None])
# print('pred', pred)
# print('true', train_labels[10])